# Image processing: PIL, imagenio, openCV
<hr>
Last version date: 12-09-2018
<hr>

## Table of Contents

1. **Initial exploration**  
2. **Open images with PIL Image**  
   2.1. Blur  
   2.2. Grayscale / RGB  
   2.3. Resize / crop  
   2.4. Enhance  
   2.5. Binarize labels  
3. **Open images with imageio**  
4. **Open images with openCV**  
   4.1. Trim borders  
   4.2. Resize  
   4.3. Calculate histogram  
   4.4. Equalize histogram  
   4.5. 2D histogram  
   4.6. CLAHE  
   4.7. Adaptive Gaussian thresholding  
   4.8. Erosion  
   4.9. Opening  
   4.10. Closing  
   4.11. Calculate image difference

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## 1. Initial exploration

In [ ]:
!ls ../input/flowers/flowers

In [ ]:
train_dir = "../input/flowers/flowers/"
classes = os.listdir(train_dir)
classes = sorted(classes)
nclasses = len(classes)
classes

In [ ]:
for _class in classes:
    print('{} {} images'.format(_class, len(os.listdir(os.path.join(train_dir, _class)))))

## 2. Open images with PIL Image

In [ ]:
from PIL import Image, ImageOps, ImageFilter

In [ ]:
other_extensions = [".db", ".pyc", ".py"]
def open_images_pil(path, classes, dim=32):
    
    xall = []
    yall = []
    label = 0
    j = 0

    for cl in classes:
        clsdir = os.path.join(path, cl)
        for imgname in os.listdir(clsdir):
            bad_ext_found = 0
            for other_ext in other_extensions:
                if imgname.endswith(other_ext):
                    bad_ext_found = 1
                    break
            if not bad_ext_found:
                print("Opening files in {}: {}".format(cl, str(j + 1)), end="\r")
                imgpath = os.path.join(clsdir, imgname)

                #open and pre-process images
                img = Image.open(imgpath)
                img = ImageOps.fit(img, (dim, dim), Image.ANTIALIAS).convert('RGB')
                
                xall.append(img)  # Get image 
                yall.append(label)  # Get image label (folder name)
                j += 1

        j = 0
        label += 1
        print()

    n = len(xall)
    print("{} images in set".format(n))
    return xall, yall

In [ ]:
xall, yall = open_images_pil(train_dir, classes, 256)

In [ ]:
im = xall[0]
im

In [ ]:
im.format, im.size, im.mode

### 2.1. Blur

In [ ]:
blur = im.filter(ImageFilter.BLUR)
blur

### 2.2. Grayscale and RGB

In [ ]:
gray = im.convert('L')
rgb = gray.convert("RGB")
gray

### 2.3. Resize / crop

In [ ]:
box = (100, 100, 400, 400)
crpd = im.crop(box)
crpd

### 2.4. Enhance

In [ ]:
enh = im.filter(ImageFilter.DETAIL)
enh

In [ ]:
_xall = np.stack(xall, axis=0) # from list of len 56 of ndarrays (dim, dim, 3) to ndarray (52, dim, dim, 3)
_xall.shape

### 2.5. Binarize label

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(yall)

In [ ]:
label = lb.transform(ori_label) 
label.shape

In [ ]:
_xall = np.asarray(xall)

for i in range(0,9): # how many imgs will show from the 3x3 grid
    plt.subplot(330 + (i+1)) # open next subplot
    plt.imshow(_xall[i + 155], cmap=plt.get_cmap('gray'))
    plt.title(yall[i + 155]);

## 3. Open images with imageio

In [ ]:
import imageio
from skimage.transform import resize

classpath = os.path.join(train_dir, classes[0])
imgpath = os.listdir(classpath)[0] # first imagepath of first class, as example
imgpath = os.path.join(classpath, imgpath)

img = imageio.imread(imgpath)
img = resize(img, (dim, dim, 3))

## 4. Open images with OpenCV (cv2)

In [ ]:
import cv2
from cv2 import imread, cvtColor, resize, threshold, calcHist, equalizeHist

### 4.1. Trim margin

In [ ]:
def trim_margin(img, lim):
    return img[lim:-lim, lim:-lim]

### 4.2. Resize

In [ ]:
supported_dims = [16, 32, 64, 128, 256]
def img_resize(img, dims):
    if dims in supported_dims:
        return cv2.resize(img, (dims, dims))
    else:
        print("Incorrect image dimensions.\n")
        return None

In [ ]:
other_extensions = [".db", ".pyc", ".py"]
def open_images_cv2(path, classes, dim=32):
    
    xall = []
    yall = []
    label = 0
    j = 0

    for cl in classes:
        clsdir = os.path.join(path, cl)
        for imgname in os.listdir(clsdir):
            bad_ext_found = 0
            for other_ext in other_extensions:
                if imgname.endswith(other_ext):
                    bad_ext_found = 1
                    break
            if not bad_ext_found:
                print("Opening files in {}: {}".format(cl, str(j + 1)), end="\r")
                imgpath = os.path.join(clsdir, imgname)

                #open and pre-process images
                img = imread(imgpath, cv2.IMREAD_COLOR)
                img = cvtColor(img, cv2.COLOR_BGR2RGB)
                img = trim_margin(img, int(img.shape[0] * 0.05))
                img = img_resize(img, dim)
                #img = equalize_hist(img)

                xall.append(img)  # Get image 
                yall.append(label)  # Get image label (folder name)
                j += 1

        j = 0
        label += 1
        print()

    n = len(xall)
    print("{} images in set".format(n))
    return xall, yall

In [ ]:
xall, yall = open_images_cv2(train_dir, classes, 256)

In [ ]:
xall = np.asarray(xall)
yall = np.asarray(yall)

In [ ]:
image = xall[0]
plt.imshow(image, cmap=plt.get_cmap('gray'))
plt.title(yall[0]);

### 4.3. Calculate histogram
```python
cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
```

* **images** : it is the source image of type uint8 or float32. it should be given in square brackets, ie, “[img]”.
* **channels** : it is also given in square brackets. It the index of channel for which we calculate histogram. For example, if input is grayscale image, its value is [0]. For color image, you can pass [0],[1] or [2] to calculate histogram of blue,green or red channel respectively.
* **mask** : mask image. To find histogram of full image, it is given as “None”. But if you want to find histogram of particular region of image, you have to create a mask image for that and give it as mask. (I will show an example later.)
* **histSize** : this represents our BIN count. Need to be given in square brackets. For full scale, we pass [256].
* **ranges** : this is our RANGE. Normally, it is [0,256].
* **[Other references][1]**

[1]: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_histograms/py_histogram_begins/py_histogram_begins.html

In [ ]:
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist([image],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

### 4.4. Equalize histogram

* **channels** = [0,1] because we need to process both H and S plane.
* **bins** = [180,256] 180 for H plane and 256 for S plane.
* **range** = [0,180,0,256] Hue value lies between 0 and 180 & Saturation lies between 0 and 256.
* **[Other references][1]**

[1]: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_histograms/py_2d_histogram/py_2d_histogram.html

In [ ]:
img_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

# equalize the histogram of the Y channel
img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

# convert the YUV image back to RGB format
imgo = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

plt.imshow(imgo)

In [ ]:
color = ('r','g','b')
for i,col in enumerate(color):
    histr = cv2.calcHist([imgo],[i],None,[256],[0,256])
    print(max(histr))
    plt.plot(histr,color=col)
    plt.xlim([0,256])
plt.show()

### 4.5. 2D histogram

In [ ]:
# https://lmcaraig.com/image-histograms-histograms-equalization-and-histograms-comparison/

from matplotlib import ticker

bins = 256
tick_spacing = 5

fig, axes = plt.subplots(1, 3, figsize=(12, 5))
channels_mapping = {0: 'B', 1: 'G', 2: 'R'}
for i, channels in enumerate([[0, 1], [0, 2], [1, 2]]):
    hist = cv2.calcHist([image], channels, None, [bins]*2, [0, 256]*2)

    channel_x = channels_mapping[channels[0]]
    channel_y = channels_mapping[channels[1]]

    ax = axes[i]
    ax.set_xlim([0, bins - 1])
    ax.set_ylim([0, bins - 1])

    ax.set_xlabel(f'Channel {channel_x}')
    ax.set_ylabel(f'Channel {channel_y}')
    ax.set_title(f'2D Color Histogram for {channel_x} and {channel_y}')

    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

    im = ax.imshow(hist)

fig.colorbar(im, ax=axes.ravel().tolist(), orientation='orizontal')
fig.suptitle(f'2D Color Histograms with {bins} bins', fontsize=16)
plt.show()

### 4.6 Contrast Limited Adaptive Histogram Equalization (CLAHE)

[From Github][1]

[1]: https://github.com/jagracar/OpenCV-python-tests/blob/master/OpenCV-tutorials/imageProcessing/histogramEqualization2.py


In [ ]:
clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(8, 8))
gray = cvtColor(image, cv2.COLOR_RGB2GRAY)
claheImg = clahe.apply(gray)
plt.imshow(claheImg)

 ### 4.7. Adaptive Gaussian thresholding
 
 [From opencv tutorial][1]
 
[1]: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html

In [ ]:
imgpath = "../input/files/files/unsplash/-537308-unsplash.jpg"

gray
img = cv2.medianBlur(gray,5)

ret, th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

# doesn't look very good

### 4.8. [Erosion][1]

[1]: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(image, kernel, iterations=1)

plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(erosion)

### 4.9. Opening
(same link)

In [ ]:
opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(opening)

### 4.10. Closing
(same link)

In [ ]:
closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(closing)

### 4.11. [Calculate image difference with opencv][1]

This value gives insight about how similar two images are.

[1]: https://www.pyimagesearch.com/2017/06/19/image-difference-with-opencv-and-python/

In [ ]:
from skimage.measure import compare_ssim
from skimage.transform import resize

image2 = xall[1]

(score, diff) = compare_ssim(image, image2, full=True, multichannel=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))